# Music21 Scale Tables

In this notebook we are developing a eye-candy presentation of scale data generated by music21 and our music21 toolchain.

## Web of Reference

### Music / Sheet Music

* music21
* abcjs

### Tables

* plim
* pandas
* d3js

### Colors

* matplotlib
  * [Pyplot colormap line by line – StackOverflow](http://stackoverflow.com/questions/27443050/pyplot-colormap-line-by-line)
  * [Choosing Colormaps – Matplotlib](http://matplotlib.org/users/colormaps.html)
  * [cm (colormap)](http://matplotlib.org/api/cm_api.html)
  * [colors – Matplotlib](http://matplotlib.org/api/colors_api.html)
* [colormap (pypi)](https://pypi.python.org/pypi/colormap)
* [palettable](https://jiffyclub.github.io/palettable) / [source](https://github.com/jiffyclub/palettable)
* [pallete (pypi)](https://pypi.python.org/pypi/palette)
* [colour (pypi)](https://pypi.python.org/pypi/colour)

### Audio

* Tone.js
* MIDI.js